In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Simulate some fake data
np.random.seed(42)
n_samples = 1000

# Features: speed (knots), cargo_weight (tons), fuel_type (0: diesel, 1: LNG)
speed = np.random.uniform(10, 25, n_samples)
cargo_weight = np.random.uniform(5000, 20000, n_samples)
fuel_type = np.random.choice([0, 1], size=n_samples)

# Simulated emissions formula
# Diesel emits more than LNG. Emissions increase with weight and speed.
emissions = (
    0.2 * speed +
    0.0005 * cargo_weight +
    (fuel_type * 0.5 + 1.0) * 10 +
    np.random.normal(0, 2, n_samples)  # noise
)

# Create DataFrame
df = pd.DataFrame({
    'speed': speed,
    'cargo_weight': cargo_weight,
    'fuel_type': fuel_type,
    'emissions': emissions
})

# Split features and label
X = df[['speed', 'cargo_weight', 'fuel_type']].values
y = df['emissions'].values

# Normalize the input features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split into train/test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Define a simple neural network model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(1)  # single output: predicted emissions
])

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=16, verbose=0)

# Evaluate
loss, mae = model.evaluate(X_test, y_test)
print(f"Test MAE: {mae:.2f} metric tons of CO₂")

# Predict for a new ship
new_ship = np.array([[18, 12000, 1]])  # 18 knots, 12,000 tons, LNG
new_ship_scaled = scaler.transform(new_ship)
predicted_emission = model.predict(new_ship_scaled)
print(f"Predicted emissions: {predicted_emission[0][0]:.2f} metric tons of CO₂")

7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 4.6184 - mae: 1.7435
Test MAE: 1.71 metric tons of CO₂
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 218ms/step
Predicted emissions: 23.49 metric tons of CO₂
